# Who is working on this project

(One person submits the document, other teammates just submit a note about who submitted the document.) Describe how you plan to work together so that everyone feels ownership of the result.

# Vision
Overview of yoru project and its purpose. what are you trying to do? Why is it important or interesting? What does a successful project outcome look like?

# Background
What data are you using? Describe what you chose and why. Include a “backup” dataset in case the primary one doesn’t work out (or give specific evidence for your confidence in the primary dataset).<br>
What technologies are you using? Briefly describe a few options you’re considering and what criteria you’ll use to evaluate them.<br>
Your final report will describe the technologies you’re using and why you chose to use them. Include citations of the work on which you’ve based your system, both what we’ve used in class and new technologies you’ve experimented with (include descriptions of these if applicable).

# Implementation
What prior code can you build on?<br>
Your final report will summarize your implementation and, if appropriate, how it extends the work you’ve reverenced.

# Prompt Engineering

In [ ]:
import torch, os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
# Work around a bug in the version of PyTorch and GPU hardware curretnly on Kaggle. On other hardware, removing these lines may lead to a speed-up.
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# Load the model
USE_INSTRUCTION_TUNED = False # we'll switch this to True partway through the lab
if USE_INSTRUCTION_TUNED:
    model_name = '/kaggle/input/gemma/transformers/1.1-2b-it/1'
    if not os.path.exists(model_name):
        print("Warning: loading model weights from the Internet. This might take a bit of extra time.")
        model_name = "google/gemma-1.1-2b-it"
else:
    model_name = "/kaggle/input/gemma/transformers/2b/2"
    if not os.path.exists(model_name):
        print("Warning: loading model weights from the Internet. This might take a bit of extra time.")
        model_name = "google/gemma-2b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    torch_dtype=torch.bfloat16)
streamer = TextStreamer(tokenizer)
# Silence a warning.
tokenizer.decode([tokenizer.eos_token_id]);

In [3]:
# Check where the whole model is loaded and what data type it's using.
model.device, model.dtype

(device(type='cpu'), torch.bfloat16)

In [4]:
# Check where parameters are loaded. If this is anything other than {'': 0}
# then probably some parts of the model got offloaded onto CPU and so will run slow.
model.hf_device_map

{'': 'cpu'}

In [5]:
essay_prompt = '''In Homegoing, Pachinko, and Stories We Tell, the authors/filmmaker choose deliberate artistic strategies to present the histories they narrate. Discuss how the literary/filmic choices of Gyasi, Lee, and Polley are part of their overall theory about how the past should be constructed. In other words, how does the way in which they present their stories intersect with what they are trying to say in their stories? Are there commonalities in their aims? If so, what are they? Are there critical differences?'''

In [7]:
print(essay_prompt)

In Homegoing, Pachinko, and Stories We Tell, the authors/filmmaker choose deliberate artistic strategies to present the histories they narrate. Discuss how the literary/filmic choices of Gyasi, Lee, and Polley are part of their overall theory about how the past should be constructed. In other words, how does the way in which they present their stories intersect with what they are trying to say in their stories? Are there commonalities in their aims? If so, what are they? Are there critical differences?


In [8]:
outline_first_paragraph = '''Please write me an introduction paragraph in essay format that follows this outline:
Thesis: Authors and filmmaker use multiple perspectives to illustrate how personal and ancestral choices shape individual narratives and identities, demonstrating their theories on how history should be constructed.
Brief introduction of works and creators: Discusses Homegoing by Yaa Gyasi, Pachinko by Min Jin Lee, and Stories We Tell by Sarah Polley, setting the stage for an exploration of their narrative techniques.
'''

In [9]:
print(outline_first_paragraph)

Please write me an introduction paragraph in essay format that follows this outline:
Thesis: Authors and filmmaker use multiple perspectives to illustrate how personal and ancestral choices shape individual narratives and identities, demonstrating their theories on how history should be constructed.
Brief introduction of works and creators: Discusses Homegoing by Yaa Gyasi, Pachinko by Min Jin Lee, and Stories We Tell by Sarah Polley, setting the stage for an exploration of their narrative techniques.



In [11]:
outline_second_paragraph = '''Please write me my first supporting paragraph about personal choices shaping narratives in essay format that follows this outline:
Main idea: Personal decisions directly shape characters' identities and futures in all three works, highlighting the authors' and filmmaker's focus on the impact of individual agency within broader historical and social contexts.
Explanation: This exploration of personal choice aligns with the creators' views that history is not merely a series of events but a complex tapestry woven from individual actions and their consequences.
Example from Pachinko: Sunja's pivotal decision to engage with Hansu, and its ramifications, showcase how personal mistakes and moral dilemmas are central to character development and plot progression.
Example from Homegoing: The character H’s experience with forced labor and subsequent physical development illustrates how personal endurance and adaptation to circumstances reflect broader historical forces like slavery and institutional racism.
Example from Stories We Tell: Harry’s narrative and his one-sided love affair demonstrate how personal perceptions can deeply influence one’s identity and the stories they choose to tell or believe.'''

In [12]:
print(outline_second_paragraph)

Please write me my first supporting paragraph about personal choices shaping narratives in essay format that follows this outline:
Main idea: Personal decisions directly shape characters' identities and futures in all three works, highlighting the authors' and filmmaker's focus on the impact of individual agency within broader historical and social contexts.
Explanation: This exploration of personal choice aligns with the creators' views that history is not merely a series of events but a complex tapestry woven from individual actions and their consequences.
Example from Pachinko: Sunja's pivotal decision to engage with Hansu, and its ramifications, showcase how personal mistakes and moral dilemmas are central to character development and plot progression.
Example from Homegoing: The character H’s experience with forced labor and subsequent physical development illustrates how personal endurance and adaptation to circumstances reflect broader historical forces like slavery and institut

In [13]:
outline_third_paragraph = '''Please write me my second supporting paragraph about influence of others and ancestors in essay format that follows this outline:
Main idea: Characters’ identities and life paths are significantly influenced by the actions and statuses of those around them and their ancestors, emphasizing the interconnectedness of personal histories within larger societal narratives.
Explanation: This theme underscores the authors' and filmmaker's perspective that individual lives are not isolated but are deeply affected by the historical and relational contexts in which they exist, echoing a broader theory that history is constructed collectively rather than singularly.
Example from Pachinko: Isak’s altruistic decision to marry Sunja provides a stark contrast to her initial dilemma, showing how benevolent actions from others can redirect an individual’s life trajectory dramatically.
Example from Homegoing: The legacy of slavery, as seen through Esi and her descendant Ness, highlights how ancestral histories cast long shadows over the lives of future generations, shaping identities and opportunities long after the original events have passed.
Example from Stories We Tell: The revelation of Sarah’s true paternity and Diane’s decisions regarding her upbringing illustrate the profound impact parental choices have on children’s identities and their understanding of family narratives.'''

In [14]:
print(outline_third_paragraph)

Please write me my second supporting paragraph about influence of others and ancestors in essay format that follows this outline:
Main idea: Characters’ identities and life paths are significantly influenced by the actions and statuses of those around them and their ancestors, emphasizing the interconnectedness of personal histories within larger societal narratives.
Explanation: This theme underscores the authors' and filmmaker's perspective that individual lives are not isolated but are deeply affected by the historical and relational contexts in which they exist, echoing a broader theory that history is constructed collectively rather than singularly.
Example from Pachinko: Isak’s altruistic decision to marry Sunja provides a stark contrast to her initial dilemma, showing how benevolent actions from others can redirect an individual’s life trajectory dramatically.
Example from Homegoing: The legacy of slavery, as seen through Esi and her descendant Ness, highlights how ancestral his

In [15]:
human_essay_first_paragraph = '''Many people believe that there is only one story going on in the world, and they are the main characters. They believe that everything should revolve around them, and everybody else in this world is a small side character. We often don't think that there are billions of different stories happening all at the same time, which can then affect our narratives through their perspectives. The authors and filmmakers of Homegoing, Pachinko, and Stories We Tell, use the perspective of many people in similar situations to demonstrate how the narratives of people’s lives can be shaped and defined by the choices they make personally, and the people around them including their ancestors. '''

In [16]:
print(human_essay_first_paragraph)

Many people believe that there is only one story going on in the world, and they are the main characters. They believe that everything should revolve around them, and everybody else in this world is a small side character. We often don't think that there are billions of different stories happening all at the same time, which can then affect our narratives through their perspectives. The authors and filmmakers of Homegoing, Pachinko, and Stories We Tell, use the perspective of many people in similar situations to demonstrate how the narratives of people’s lives can be shaped and defined by the choices they make personally, and the people around them including their ancestors. 


In [17]:
human_essay_second_paragraph = '''People’s identities are often defined by the choices they made in the past, whether they are successful or unsuccessful, honorable or dishonorable. The authors and filmmakers of these stories demonstrate how a person’s choices can shape their narratives. In the novel, Pachinko by Min Jin Lee, we mainly see the story through the viewpoint of Sunja, but we will occasionally get to look through or see the thoughts of the people close to her. In the novel, it states, “If he did not marry her, she was a common slut who would be disgraced forever. The child would be another no-name bastard. Her mother’s boardinghouse would be contaminated by her shame”(Lee 49). Sunja made a big mistake by being with Hansu, and she paid the price. She was now pregnant, and the father of the child won’t be able to marry her because he is already married. In her society, bearing a child without a father will lead to having the mother, in this case, Sunja, be disowned. It could also lead to her family and her child being disowned as well. Looking through the lens of Sunja, we could see how devastated she was and the impact her decisions could make on her in the future. Also, in the novel, Homegoing by Yaa Gyasi, we see through many different lenses in many different generations. In the chapter about H, it states, “The boss man was called Mr. John. He asked to take off his shirt. He inspected the muscles on his back and his arms and whistled. ‘Any man what can spend ten years working at Rock Slope and live to tell about it’s worth watching”(Gyasi 169). When H was sent to me while he was arrested, he might not have had an option on whether or not he wanted to work, but it made him better in the long run. He had gotten physically strong, and when he had gotten released, he was able to find a mining job that paid him. The situation he was put in shaped the person that H was. He became a hard worker and he made sure he did his job. Those were some of the things that he learned while he mined for the jail, but he was able to apply those lessons to the real world and shaped him to be somebody that was hirable. Another example of identities being defined by self choices is in the documentary, Stories We Tell, produced by Sarah Polley. When Harry met with Diane when she traveled to Montreal for a play, Harry fell in love with her. He had his own one-sided story, where he thought he was going to be with Diane for the rest of his life. This ultimately shaped his identity for the future. At the end of the documentary, Sarah asked Harry whether he liked that many people are sharing their sides of the story or if he disliked it. He shared that he did not like how many people shared their sides because he thought it was only his story to tell. He got wrapped up in the sense that Diane only loved him, and that changed the way he viewed his story.'''

In [18]:
print(human_essay_second_paragraph)

People’s identities are often defined by the choices they made in the past, whether they are successful or unsuccessful, honorable or dishonorable. The authors and filmmakers of these stories demonstrate how a person’s choices can shape their narratives. In the novel, Pachinko by Min Jin Lee, we mainly see the story through the viewpoint of Sunja, but we will occasionally get to look through or see the thoughts of the people close to her. In the novel, it states, “If he did not marry her, she was a common slut who would be disgraced forever. The child would be another no-name bastard. Her mother’s boardinghouse would be contaminated by her shame”(Lee 49). Sunja made a big mistake by being with Hansu, and she paid the price. She was now pregnant, and the father of the child won’t be able to marry her because he is already married. In her society, bearing a child without a father will lead to having the mother, in this case, Sunja, be disowned. It could also lead to her family and her ch

In [19]:
human_essay_third_paragraph = '''While some believe that their identity is solely based on their choices, people’s identities can be altered by the people around them and by their ancestors. When we start looking through the other lenses of people in the same situation, we can see that many times the expected outcome is different from reality. In the novel, Pachinko, it states, “‘Of course it would be far better for them if she went away’ Yangjin replied, knowing the hard truth. ‘The child would have a terrible life here. You’d be saving my daughter’s life as well'”(Lee 74). Isak knew the dilemma Sunja was in and he knew this was something that shouldn’t be taken lightly. After coming up with the idea of marrying Sunja and giving the child his last name, he had lifted the burden off Sunja and her child’s back. They would no longer have to bear the weight of being dishonored by society and her family. Having people who can have a different perspective of the matter, can allow one to alter the course of somebody. Then, in the novel, Homegoing, we were able to witness the identities being altered because of their past relatives. In the novel, it states, “Every day, Ness picked cotton under the punishing eye of the southern sun. She had been at Thomas Allan Stockham’s Alabama plantation for three months”(Gyasi 70). Ness was a descendant of Esi in this story. Esi was a part of the slave trade system and her life was very rough. Esi went through an enormous amount of abuse and unbearable situations. Since being in that slave system, it had affected the rest of her family tree. Ness is the daughter of Esi, and so, Ness had to bear the identity of being a slave just like her mother. She had to be a slave on a plantation and no choice she made would be able to change that in this system. Her identity at the time was determined by her mother and how she was a part of the system. Lastly, in the documentary, Stories We Tell, Sarah’s identity was shaped by the choices her mother made when she was alive. Sarah never knew that Michael was not her biological father until she grew up. Sarah was able to form a strong bond with Michael, and that was because Diane decided to have Michael take care of her instead of Harry. Once Sarah found out that Harry was her father, Sarah was still able to keep the close relationship with Michael even after finding out the truth. Sarah was able to keep her identity from the one she formed with Michael, and that was all due to Diane.'''

In [20]:
print(human_essay_third_paragraph)

While some believe that their identity is solely based on their choices, people’s identities can be altered by the people around them and by their ancestors. When we start looking through the other lenses of people in the same situation, we can see that many times the expected outcome is different from reality. In the novel, Pachinko, it states, “‘Of course it would be far better for them if she went away’ Yangjin replied, knowing the hard truth. ‘The child would have a terrible life here. You’d be saving my daughter’s life as well'”(Lee 74). Isak knew the dilemma Sunja was in and he knew this was something that shouldn’t be taken lightly. After coming up with the idea of marrying Sunja and giving the child his last name, he had lifted the burden off Sunja and her child’s back. They would no longer have to bear the weight of being dishonored by society and her family. Having people who can have a different perspective of the matter, can allow one to alter the course of somebody. Then, 

In [35]:
'''
def generate_and_analyze(outline, essay):
    # Generate text based on the outline
    model_input = tokenizer.encode(outline, return_tensors='pt').to(model.device)
    generated_output = model.generate(model_input, max_length=512, num_return_sequences=1)
    generated_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)
    
    # Tokenize the generated text and the human essay for comparison
    gen_ids = tokenizer.encode(generated_text, return_tensors='pt').to(model.device)
    essay_ids = tokenizer.encode(essay, return_tensors='pt').to(model.device)
    
    # Concatenate tokens for analysis
    input_ids = torch.cat([gen_ids, essay_ids[:, 1:]], dim=-1)
    
    # Generate logits for concatenated input
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits
    
    # Analyze tokens for loss
    # Analyze tokens for loss
    spans = []
    highest_loss = float('-inf')
    softmax = torch.nn.Softmax(dim=-1)
    essay_tokens = tokenizer.convert_ids_to_tokens(essay_ids[0])

    for i in range(gen_ids.size(1), input_ids.size(1)):
        probs = softmax(logits[0, i - 1])
        token_loss = -torch.log(probs[input_ids[0, i]]).item()
        most_likely_token_id = torch.argmax(probs).item()
        token = essay_tokens[i - gen_ids.size(1) + 1]  # Adjust index for essay tokens
        most_likely_token = tokenizer.decode([most_likely_token_id])
    
        spans.append({
            'original_token': token,
            'token_loss': token_loss,
            'most_likely_token': most_likely_token,
            'loss_ratio': None  # Will be calculated below
        })
        if token_loss > highest_loss:
            highest_loss = token_loss

    # Normalize loss ratios
    for span in spans:
        span['loss_ratio'] = span['token_loss'] / highest_loss

    
    df = pd.DataFrame(spans)
    return {
        'outline': outline,
        'generated_text': generated_text,
        'analysis_df': df
    }
'''

"\ndef generate_and_analyze(outline, essay):\n    # Generate text based on the outline\n    model_input = tokenizer.encode(outline, return_tensors='pt').to(model.device)\n    generated_output = model.generate(model_input, max_length=512, num_return_sequences=1)\n    generated_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)\n    \n    # Tokenize the generated text and the human essay for comparison\n    gen_ids = tokenizer.encode(generated_text, return_tensors='pt').to(model.device)\n    essay_ids = tokenizer.encode(essay, return_tensors='pt').to(model.device)\n    \n    # Concatenate tokens for analysis\n    input_ids = torch.cat([gen_ids, essay_ids[:, 1:]], dim=-1)\n    \n    # Generate logits for concatenated input\n    with torch.no_grad():\n        outputs = model(input_ids)\n        logits = outputs.logits\n    \n    # Analyze tokens for loss\n    # Analyze tokens for loss\n    spans = []\n    highest_loss = float('-inf')\n    softmax = torch.nn.Softmax(dim=

# Trial by just providing the outline and human written essay

In [29]:
def generate_text(outline):
    # Encode the outline and generate text
    model_input = tokenizer.encode(outline, return_tensors='pt').to(model.device)
    generated_output = model.generate(model_input, max_length=512, num_return_sequences=1)
    generated_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)
    
    # Tokenize the generated text for analysis
    gen_ids = tokenizer.encode(generated_text, return_tensors='pt').to(model.device)
    
    return generated_text, gen_ids

In [30]:
def analyze_text(generated_ids, essay):
    # Tokenize the essay for comparison
    essay_ids = tokenizer.encode(essay, return_tensors='pt').to(model.device)
    
    # Concatenate tokens for analysis
    input_ids = torch.cat([generated_ids, essay_ids[:, 1:]], dim=-1)
    
    # Generate logits for concatenated input
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits
    
    # Analyze tokens for loss
    spans = []
    highest_loss = float('-inf')
    softmax = torch.nn.Softmax(dim=-1)
    essay_tokens = tokenizer.convert_ids_to_tokens(essay_ids[0])
    
    for i in range(generated_ids.size(1), input_ids.size(1)):
        probs = softmax(logits[0, i - 1])
        token_loss = -torch.log(probs[input_ids[0, i]]).item()
        most_likely_token_id = torch.argmax(probs).item()
        token = essay_tokens[i - generated_ids.size(1) + 1]  # Adjust index for essay tokens
        most_likely_token = tokenizer.decode([most_likely_token_id])
        
        spans.append({
            'original_token': token,
            'token_loss': token_loss,
            'most_likely_token': most_likely_token,
            'loss_ratio': None  # To be calculated later
        })
        
        if token_loss > highest_loss:
            highest_loss = token_loss

    # Normalize loss ratios
    for span in spans:
        span['loss_ratio'] = span['token_loss'] / highest_loss

    df = pd.DataFrame(spans)
    return df

In [31]:
outline = outline_first_paragraph
essay = human_essay_first_paragraph

# Generate text based on the outline
generated_text, generated_ids = generate_text(outline)

# Analyze the generated text against the provided essay
analysis_df = analyze_text(generated_ids, essay)

# Display results
print("Generated Essay:", generated_text)
print("\nAnalysis:")
display(analysis_df[['original_token', 'token_loss', 'most_likely_token', 'loss_ratio']])

KeyboardInterrupt: 

In [ ]:
def rewrite_essay(analysis_df, essay_text):
    # Tokenize the original essay text
    essay_ids = tokenizer.encode(essay_text, return_tensors='pt').to(model.device)
    essay_tokens = tokenizer.convert_ids_to_tokens(essay_ids[0])

    # Create a new list to hold the rewritten essay tokens
    rewritten_tokens = []
    
    for idx, row in analysis_df.iterrows():
        # Ensure index alignment with essay tokens
        if idx < len(essay_tokens):
            # Replace the token if the loss ratio is above a certain threshold
            if row['loss_ratio'] > 0.5:  # This threshold can be adjusted
                rewritten_tokens.append(row['most_likely_token'])
            else:
                rewritten_tokens.append(row['original_token'])

    # Join the tokens back into a single string using the tokenizer to ensure correct spacing
    rewritten_essay = tokenizer.convert_tokens_to_string(rewritten_tokens)
    return rewritten_essay

In [ ]:
print("Original Essay:", essay)
print("Generated Essay:", generated_text)
print("Rewritten Essay:", rewritten_essay)
print("\nAnalysis:")
display(analysis_df[['original_token', 'token_loss', 'most_likely_token', 'loss_ratio']])

In [ ]:
def highlight_replacements(analysis_df, essay_text):
    # Tokenize the original essay text
    essay_ids = tokenizer.encode(essay_text, return_tensors='pt').to(model.device)
    essay_tokens = tokenizer.convert_ids_to_tokens(essay_ids[0])

    replacements = []
    
    # Check for replacements in the analysis dataframe
    for idx, row in analysis_df.iterrows():
        if idx < len(essay_tokens):
            original_token = row['original_token']
            replacement_token = row['most_likely_token']
            
            # Check if replacement is different and loss ratio is above threshold
            if original_token != replacement_token and row['loss_ratio'] > 0.5:
                replacements.append((original_token, replacement_token))
    
    return replacements

In [ ]:
outline = outline_first_paragraph
essay = human_essay_first_paragraph
generated_text, generated_ids = generate_text(outline)
analysis_df = analyze_text(generated_ids, essay)

# Get the list of replacements
replacements = highlight_replacements(analysis_df, essay)

In [ ]:
print("Original Essay:", essay)
print("Rewritten Essay:", rewritten_essay)
print("\nReplacements:")
for original, replaced in replacements:
    print(f"Original: '{original}' was replaced with '{replaced}'")

# Trial by including the prompt

In [ ]:
def generate_text_with_prompt(prompt, outline):
    full_input = prompt + " " + outline
    model_input_with_prompt = tokenizer.encode(full_input, return_tensors='pt').to(model.device)
    generated_output_with_prompt = model.generate(model_input, max_length=512, num_return_sequence=1)
    generated_text_with_prompt = tokenizer.decode(generated_output[0], skip_special_tokens=True)
    
    gen_ids_with_prompt = tokenizer.encode(generated_text, return_tensors='pt').to(model.device)
    
    return generated_text_with_prompt

In [ ]:
def analyze_text_with_prompt(generated_ids, essay):
    essay_ids_with_prompt = tokenizer.encode(essay, return_tensors='pt').to(model.device)
    input_ids = torch.cat([generated_ids, essay_ids[:, 1:]], dim=-1)
    
    with torch.no_grad():
        outputs = model(input_ids)
        logits = output.logits
        
    spans = []
    highest_loss = float('-inf')
    softmax = torch.nn.Softmax(dim=-1)
    essay_tokens = tokenizer.convert_ids_to_tokens(essay_ids[0])
    
    for i in range(generated_ids.size(1), input_ids.size(1)):
        probs = softmax(logits[0, i-1])
        token_loss = -torch.log(probs[input_ids[0,i]]).item()
        most_likely_token_id = torch.argmax(probs).item()
        token = essay_tokens[i - generated_ids.size(1) + 1]
        most_likely_token = tokenizer.decode([most_likely_token_id])
        
        spans.append({
            'original_token' : token,
            'token_loss' : token_loss,
            'most_likely_token' : most_likely_token,
            'loss_ratio' : None
        })
        
        if token_loss > highest_loss:
            highest_loss = token_loss
        
        for span in spans:
            span['loss ratio'] = span['token_loss'] / highest_loss
            
        return pd.DataFrame(spans)

In [ ]:
prompt = prompt
outline = outline_first_paragraph
essay = human_essay_first_paragraph

generated_text, generated_ids = generate_text_with_prompt(prompt, outline)
analysis_df = analyze_text_with_prompt(generated_ids, essay)

In [ ]:
print("Prompt:", prompt)
print("Original Essay:", essay)
print("\nGenerated Essay:", generated_text)
print("\nRewritten Essay:", rewritten_essay)

# Results
 Include quantitative (tables, plots) and qualitative (examples) results, including comparisons with similar work if applicable.

# Implications
Discuss the social and ethical implications of using the technologies you’ve chosen for your project.